In [2]:
!gcloud dataproc clusters list --region us-central1
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes
import pyspark
import math
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *
import builtins
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter
import re
from math import log



import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')
nltk.download('punkt')

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-38ae  GCE       2                                             RUNNING  us-central1-a


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
from time import time
from pathlib import Path
import pickle
from google.cloud import storage
from collections import defaultdict
from contextlib import closing

PROJECT_ID = 'triple-bird-414408'

def get_bucket(bucket_name):
    return storage.Client(PROJECT_ID).bucket(bucket_name)

def _open(path, mode, bucket=None):
    if bucket is None:
        return open(path, mode)
    return bucket.blob(path).open(mode)


TUPLE_SIZE = 6       # We're going to pack the doc_id and tf values in this 
                     # many bytes.
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer

In [4]:
%cd -q /home/dataproc
!ls inverted_index_gcp.py
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())
from inverted_index_gcp import InvertedIndex

inverted_index_gcp.py


In [9]:
# %cd -q /home/dataproc
# !ls search_backend.py
# # adding our python module to the cluster
# sc.addFile("/home/dataproc/search_backend.py")
# sys.path.insert(0,SparkFiles.getRootDirectory())
# # from search_backend import *

In [5]:
#load the paths of the preprocessed data from the bucket
bucket_name = '204800122_wikidata'
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh' and b.name != 'init_file.sh':
        paths.append(full_path+b.name)
        
#Take only 1 file for testing
paths_2 = paths[:1]
corpus = spark.read.parquet(*paths_2)
corpus_len = corpus.count()

In [6]:
# Update your stopwords setup as before
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = set([word.lower() for word in english_stopwords.union(corpus_stopwords)])

ps = PorterStemmer()

token_pattern = re.compile(r"""[\w](['\-]?\w)*""")

def get_tokens(text, stem=False, bigrams=False, trigrams=False):
    tokens = word_tokenize(text.lower())
    filtered_tokens = [token for token in tokens if token not in all_stopwords and token_pattern.match(token)]
    
    # Apply stemming if requested
    if stem:
        filtered_tokens = [ps.stem(token) for token in filtered_tokens]
    
    # Generate bigrams if requested
    if bigrams:
        filtered_tokens = [f"{filtered_tokens[i]} {filtered_tokens[i+1]}" for i in range(len(filtered_tokens)-1)]
        
    # Generate trigrams if requested
    if trigrams:
        # This will create a list of trigrams where each trigram is represented as a single string
        filtered_tokens = [f"{filtered_tokens[i]} {filtered_tokens[i+1]} {filtered_tokens[i+2]}" for i in range(len(filtered_tokens)-2)]
    
    return filtered_tokens

def word_count(text, id,stem=False,bigrams=False, trigrams=False):
    # Use get_tokens to process the text
    tokens = get_tokens(text,stem,bigrams,trigrams)
    return [(term, (id, tf)) for term, tf in Counter(tokens).items()]


NUM_BUCKETS = 124
def token2bucket_id(token):
    return int(_hash(token),16) % NUM_BUCKETS

def calculate_df(postings):
    return postings.map(lambda x: (x[0], len(x[1])))

def reduce_word_counts(unsorted_pl):
    return sorted(unsorted_pl, key=lambda x: x[0], reverse=True)


def partition_postings_and_write(postings, bucket_name):
    print(bucket_name)
    return postings.map(lambda x: (token2bucket_id(x[0]), (x[0], x[1]))).groupByKey().mapValues(list).map(lambda x: InvertedIndex.write_a_posting_list(b_w_pl=x, base_dir='', bucket_name=bucket_name))

def calc_tdidf(text, id, df, corpus_size,stem=False,bigrams=False, trigrams=False):
    """without stemming"""
    tokens = get_tokens(text,stem,bigrams,trigrams)
    words_counter = Counter(tokens)
    size = builtins.sum([(c/len(tokens)*math.log2(corpus_size/df[w]))**2 for w,c in words_counter.items() if w in df])
    return (id, math.sqrt(size))


def anchors_to_pairs(row):
  pairs =[]
  for doc_id, anchor in row[0]:
    pairs.append((doc_id, anchor))
  return pairs

In [7]:
def create_index(index_name,corpus_len,document_part,corpus,stem=False,bigrams=False,trigrams=False):
    index_name = f"{index_name}"        
    doc_pairs = corpus.select(document_part, "id").rdd
    word_counts = doc_pairs.flatMap(lambda x: word_count(x[0], x[1],stem,bigrams,trigrams))
    postings = word_counts.groupByKey().mapValues(reduce_word_counts)
    postings_filtered = postings.filter(lambda x: len(x[1])>50)
    term_total = postings_filtered.map(lambda x: (x[0],builtins.sum([i[1] for i in x[1]])))
    doc_len = doc_pairs.map(lambda x: (x[1], len(get_tokens(x[0],stem,bigrams,trigrams))))
    w2df = calculate_df(postings_filtered)
    w2df_dict = w2df.collectAsMap()
    _ = partition_postings_and_write(postings_filtered, bucket_name).collect()
    super_posting_locs = defaultdict(list)
    for blob in client.list_blobs(bucket_name, prefix=''):
        if not blob.name.endswith("pickle"):
            continue
        with blob.open("rb") as f:
            posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            super_posting_locs[k].extend(v)
    # Create inverted index instance
    inverted = InvertedIndex()
    # Adding the posting locations dictionary to the inverted index
    inverted.posting_locs = super_posting_locs
    # Add the token - df dictionary to the inverted index
    inverted.df = w2df.collectAsMap()
    # Add the Document leangth -> use for bm25
    inverted.document_len = doc_len.collectAsMap()
    # Add the term_total
    inverted.term_total = term_total.collectAsMap()
    #calculate tdidf
    tf_score = doc_pairs.map(lambda x: calc_tdidf(x[0], x[1], w2df_dict, corpus_len,stem,bigrams,trigrams))
    inverted.document_size = dict(tf_score.collect())
    # write the global stats out
    inverted.write_index('.', f'index_{index_name}')

    # upload to gs
    index_src = f"index_{index_name}.pkl"
    index_dst = f'gs://{bucket_name}/{index_src}'
    !gsutil cp $index_src $index_dst

    print(index_src)

In [15]:
bucket_name = "anchor_index_nostem"
doc_anchor = corpus.select("anchor_text").rdd
inverted_anchor = InvertedIndex()
doc_anchor_pairs = doc_anchor.flatMap(anchors_to_pairs).groupByKey().mapValues(list).map(lambda x: (" ".join(x[1]), x[0]))
word_counts = doc_anchor_pairs.flatMap(lambda x: word_count(x[0], x[1],stem=False,bigrams=False,trigrams=False))
doc_len = doc_anchor_pairs.map(lambda x: (x[1], len(get_tokens(x[0],stem=False,bigrams=False,trigrams=False))))

In [16]:
len_docs_anchor = doc_len.collectAsMap()
inverted_anchor.document_len = len_docs_anchor

In [17]:
from operator import add
postings_anchor = word_counts.groupByKey().mapValues(reduce_word_counts)
total_terms_anchor = postings_anchor.flatMapValues(lambda x : x).map(lambda x: (x[0],x[1][1])).reduceByKey(add)
inverted_anchor.term_total = total_terms_anchor.collectAsMap()

In [18]:
w2df_anchor = calculate_df(postings_anchor)
w2df_anchor_dict = w2df_anchor.collectAsMap()
inverted_anchor.df = w2df_anchor_dict

In [19]:
_ = partition_postings_and_write(postings_anchor,"anchor_index_nostem").collect()

anchor_index_nostem


In [13]:
# tf_score = doc_anchor_pairs.map(lambda x: calc_tdidf(x[0], x[1], w2df_anchor_dict, corpus_len,stem=False,bigrams=False,trigrams=False))
# inverted_anchor.document_size = dict(tf_score.collect())

In [20]:
# collect all posting lists locations into one super-set
super_posting_locs_anchor = defaultdict(list)
for blob in client.list_blobs(bucket_name):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            super_posting_locs_anchor[k].extend(v)

#stores it in anchor_index.posting_locs attribute
inverted_anchor.posting_locs = super_posting_locs_anchor
inverted_anchor.write_index('.','index_anchor_nostem')

# upload to gs
index_src = "index_anchor_nostem.pkl"
index_dst = f'gs://anchor_index_nostem/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://index_anchor_nostem.pkl [Content-Type=application/octet-stream]...
- [1 files][ 43.2 MiB/ 43.2 MiB]                                                
Operation completed over 1 objects/43.2 MiB.                                     


24/02/26 17:34:00 WARN BlockManagerMasterEndpoint: No more replicas available for broadcast_13_python !


In [57]:
# def anchor_index_creation(index_name,corpus_len,corpus):
#     index_name = f'{index_name}'   
#     # Convert the corpus DataFrame into an RDD of anchor text and document ID pairs
#     anchor_pairs = corpus.select("anchor_text", "id").rdd
    
#     # Manipulate the tuples so each tuple contains (text, the doc_id of the document we found the anchor_text)
#     postings_not_grouped = anchor_pairs.flatMap(lambda row: [(anchor.text, row.id) for anchor in row.anchor_text])
#     doc_len = doc_pairs.map(lambda x: (x[1], len(get_tokens(x[0],stem,bigrams,trigrams))))
#     postings = anchor_pairs.flatMap(lambda row: [(anchor.text, row.id) for anchor in row.anchor_text]).groupByKey()
#     anchor_posting = postings.mapValues(list).map(lambda x: (x[0], Counter(x[1]).most_common()))
    
#     # Filter postings to include only those with more than 0 occurrences
#     postings_filtered = anchor_posting.filter(lambda x: len(x[1]) > 0)  # ("word", {doc_id1: number of appearances})
    
#     # Create an inverted index instance
#     anchor_InvertedIndex = InvertedIndex()
    
#     # Calculate document frequency and update the inverted index
#     anchor_InvertedIndex.df = dict(Counter(calculate_df(postings_filtered).collectAsMap()))
    
#     # Partition postings and write them to the specified bucket
#     _ = partition_postings_and_write(postings_filtered, bucket_name).collect()
    
#     # Collect all posting lists locations into one super-set
#     super_posting_locs_text = defaultdict(list)
#     for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
#         if not blob.name.endswith("pickle"):
#             continue
#         with blob.open("rb") as f:
#             posting_locs = pickle.load(f)
#             for k, v in posting_locs.items():
#                 super_posting_locs_text[k].extend(v)
                
#     # Update the inverted index with postings locations
#     anchor_InvertedIndex.posting_locs = super_posting_locs_text
#     anchor_InvertedIndex.document_len = doc_len.collectAsMap()
#     # Add the term_total
#     anchor_InvertedIndex.term_total = term_total.collectAsMap()
#     #calculate tdidf
#     tf_score = postings_not_grouped.map(lambda x: calc_tdidf(x[0], x[1], w2df_dict, corpus_len,stem,bigrams,trigrams))
#     anchor_InvertedIndex.document_size = dict(tf_score.collect())
#     anchor_InvertedIndex.write_index('.', f'index_{index_name}')

#     # upload to gs
#     index_src = f"index_{index_name}.pkl"
#     index_dst = f'gs://{bucket_name}/{index_src}'
#     !gsutil cp $index_src $index_dst


In [60]:
bucket_name = "title_index_nostem"

index_location = create_index("title_index_withStem_bigrams", corpus_len, "title", corpus, stem=False, bigrams=False, trigrams=False)

title_index_nostem


Copying file://index_title_index_withStem_bigrams.pkl [Content-Type=application/octet-stream]...
/ [1 files][  3.4 MiB/  3.4 MiB]                                                
Operation completed over 1 objects/3.4 MiB.                                      
index_title_index_withStem_bigrams.pkl


In [64]:
bucket_name = "text_index_nostem"
index_location = create_index("text_index_nostem", corpus_len, "text", corpus, stem=False, bigrams=False, trigrams=False)

text_index_nostem


Copying file://index_text_index_nostem.pkl [Content-Type=application/octet-stream]...
/ [1 files][  5.8 MiB/  5.8 MiB]                                                
Operation completed over 1 objects/5.8 MiB.                                      
index_text_index_nostem.pkl


In [62]:
pairs = corpus.select("id", "title").rdd

# Convert RDD to a Python dictionary
id_to_title_dict = dict(pairs.collect())

# Specify the path where you want to save the pickle file
pickle_file_name = "id_to_title_dict.pkl"
bucket_name = "title_index_nostem"
client = storage.Client()
bucket = client.bucket(bucket_name)
blob = bucket.blob(pickle_file_name)
pickle_bytes = pickle.dumps(id_to_title_dict)
blob.upload_from_string(pickle_bytes)